In [ ]:
import math
import glob
import os

import numpy as np
from tqdm import tqdm
import pandas as pd


SYMBOLS = ["AAPL"]

DATADIR = os.path.join(os.getcwd(), 'historical-market-data')

for symbol in tqdm(SYMBOLS, desc="Symbols"):
    path = f'{DATADIR}/{symbol}-2016-*.csv'
    days = sorted(glob.glob(path))

    first_day = pd.read_csv(days[0])
    last_day = pd.read_csv(days[-1])

    last_price = last_day.iloc[-1]["close"]
    initial_price = first_day.iloc[-1]["close"]



In [ ]:
owned = False
num_transactions = 0
df_prev = None

In [ ]:
purchases_day = []
print(purchases_day)
sells_day = []
print(sells_day)

for d, day in enumerate(tqdm(days, desc=f"{symbol:4} days")):

    print(d, day)

    df = pd.read_csv(day)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True, drop=False)
    df = df.between_time('09:30', '16:00')

    if df_prev is None:
        df_prev = df
        continue


    prev_open_stddev = df_prev['open'].rolling(10).std().mean()
    print(prev_open_stddev)
    df['prev_open_stddev'] = prev_open_stddev

    
    avg_window = 4
    df['m_avg'] = df['close'].rolling(avg_window).mean()


    force_sell = False


    df.reset_index(inplace=True, drop=True)
    for i, row in df.iterrows():
        open_price = row["open"]


        if day == days[-1] and row.timestamp == df.iloc[-1].timestamp:
            final_price = row["close"]
            if owned is True:
                force_sell = True
                print('forced sell')


        lookback = i - 2
        if lookback < 0 or math.isnan(df.iloc[lookback]['m_avg']):
            continue


        if owned is False:


            buy_signal = (
                df.iloc[i - 2]['close'] < df.iloc[i - 2]['m_avg'] - prev_open_stddev
                and 
                df.iloc[i - 1]['close'] > df.iloc[i - 1]['m_avg']
            )

            if buy_signal:
                price = open_price
                buy_price = price

                # increase transaction counter
                num_transactions += 1

                # add transaction to our list
                purchases_day.append([row['timestamp'], price])

                # change the owned flag to True
                owned = True

        elif owned is True:

            # algo sells if current open_price > buy_price,
            # or if forced_sell is True
            sell_signal = (
                open_price > buy_price + prev_open_stddev
                or force_sell is True
            )

            if sell_signal:
                price = open_price
                sell_price = price
                num_transactions += 1
                sells_day.append([row['timestamp'], price])
                owned = False

    df_prev = df


print(initial_price)
print(last_price)

col = 0
for i, v in zip(purchases_day, sells_day):
    col += v[-1] - i[-1]

print(col)
print(f"{symbol} - num_transactions: {num_transactions}")
print(f'{initial_price}, {last_price}, {last_price - initial_price}, {col}')